## TFlearn

TFLearn creates a TrainOp class to describe an optimization procedure (such as backprop). 
- Here's how it's defined:

# DO NOT RUN ANY CODE

In [4]:
import tensorflow as tf

In [2]:
# import tflearn
# it will throw import error because of the following reason
# I believe it is because tflearn==0.5.0 which is too old and cannot work with latest tensorflow=2.13.0

In [ ]:
#Define your network architecture
input_placeholder = tf.placeholder(tf.float32, shape=[None, input_size]) 
target_placeholder = tf.placeholder(tf.float32, shape=[None, num_classes]) 
my_network = tflearn.fully_connected(input_placeholder, 32) 
loss = tflearn.objectives.categorical_crossentropy(my_network, target_placeholder) 
accuracy = tflearn.metrics.accuracy(my_network, target_placeholder)

In [ ]:
#Create TrainOp and Trainer 
trainop = tflearn.TrainOp(net=my_network, loss=loss, metric=accuracy) 
model = tflearn.Trainer(train_ops=trainop, tensorboard_dir='/tmp/tflearn')

In [ ]:
#Train the model 
model.fit(feed_dicts={input_placeholder: X, target_placeholder: Y}, 
n_epoch=10, batch_size=128, show_metric=True)

In [ ]:
#Create TrainOp objects for each training operation 
trainop1 = tflearn.TrainOp(net=network1, loss=loss1) 
trainop2 = tflearn.TrainOp(net=network2, loss=loss2) 
trainop3 = tflearn.TrainOp(net=network3, loss=loss3)

In [ ]:
#Create Trainer with multiple TrainOps
model = tflearn.Trainer(train_ops=[trainop1, trainop2, trainop3])

In [ ]:
#Train the model with different feed dictionaries for each training operation
feed_dict1 = {in1: X1, label1: Y1} 
feed_dict2 = {in2: X2, in3: X3, label2: Y2} 
model.fit(feed_dicts=[feed_dict1, feed_dict2])

###### For prediction, TFLearn implements an Evaluator class that works the same as the trainer. It takes a parameter and returns the predicted value.

In [ ]:
#Create Evaluator 
model = tflearn.Evaluator(network)

In [ ]:
#Make predictions 
predictions = model.predict(feed_dict={input_placeholder: X})

###### The Trainer class in TFLearn utilizes the is_training boolean variable to handle network behavior during training, testing, and prediction. 

In [ ]:
#Example for Dropout:
x = ...
def apply_dropout(): #Function to apply when training mode ON.
return tf.nn.dropout(x, keep_prob)
is_training = tflearn.get_training_mode() #Retrieve is_training variable.
tf.cond(is_training, apply_dropout, lambda: x) #Only apply dropout at training time.

###### TFLearn implements functions to retrieve that variable or change its value:

In [ ]:
#Set training mode ON (set is_training var to True) 
tflearn.is_training(True) 
#Set training mode OFF (set is_training var to False)
tflearn.is_training(False)

###### 

## Weight Persistence

It is the process of saving and loading learned parameter values (weights) of a trained model for 
future use or deployment

In [ ]:
#Save the weights
model.save_weights('my_model_weights.npy’)
#Load a model
torch.save(model.state_dict(), 'my_model_weights.pth’)

A layer variable can be directly retrieved by using the layer name, or indirectly by using the W or b
attributes that are associated with the layer's returned tensor.

In [ ]:
# Let's create a layer
fc1 = fully_connected(input_layer, 64, name="fc_layer_1")

In [ ]:
# Using Tensor attributes (Layer will supercharge the returned Tensor with 
weights attributes)
fc1_weights_var = fc1.W
fc1_biases_var = fc1.b

In [ ]:
# Using Tensor name
fc1_vars = tflearn.get_layer_variables_by_name("fc_layer_1")
fc1_weights_var = fc1_vars[0]
fc1_biases_var = fc1_vars[1]

The TFLearn model classes implement the get_weights and set_weights methods to obtain or set the 
values of these variables:

In [ ]:
input_data = tflearn.input_data(shape=[None, 784])
fc1 = tflearn.fully_connected(input_data, 64)
fc2 = tflearn.fully_connected(fc1, 10, activation='softmax')
net = tflearn.regression(fc2)
model = DNN(net)

In [ ]:
# Get weights values of fc2
model.get_weights(fc2.W)

In [ ]:
# Assign new random weights to fc2
model.set_weights(fc2.W, numpy.random.rand(64, 10))

###### The following is an illustration of how to save, restore, and retrieve weights for models:

pip install tflearn

In [ ]:
from __future__ import absolute_import, division, print_function
import tflearn
import tflearn.datasets.mnist as mnist

In [ ]:
# MNIST Data
x, y, testx, testy = mnist.load_data(one_hot=True)

In [ ]:
# Model
input_layer = tflearn.input_data(shape=[None, 784], name='input')
dense1 = tflearn.fully_connected(input_layer, 128, name='dense1')
dense2 = tflearn.fully_connected(dense1, 256, name='dense2')
softmax = tflearn.fully_connected(dense2, 10, activation='softmax')
regression = tflearn.regression(softmax, optimizer='adam',
learning_rate=0.001,
loss='categorical_crossentropy')

###### A classifier model with a model checkpoint that automatically saves the model and weights for later use or evaluation during training

In [ ]:
# Define classifier, with model checkpoint (autosave)
model = tflearn.DNN(regression, checkpoint_path='model.tfl.ckpt’)

In [ ]:
# Train model, with model checkpoint every epoch and every 200 training steps.
model.fit(x, y, n_epoch=1, 
validation_set=(testx, testy), 
show_metric=True, 
snapshot_epoch=True, # Snapshot (save & evaluate) model every epoch. 
snapshot_step=500, # Snapshot (save & evalaute) model every 500 steps.
run_id='model_and_weights’)

In [ ]:
# Manually save model
model.save("model.tfl")
# Load a model
model.load("model.tfl")

###### The following is an illustration of how to retrieve and print the weights:

In [ ]:
# Retrieve a layer weights, by layer name:
dense1_vars = 
tflearn.variables.get_layer_variables_by_name('dense1’)

In [ ]:
# Get a variable's value, using model `get_weights` 
method:
print("Dense1 layer weights:")
print(model.get_weights(dense1_vars[0]))

In [ ]:
# Or using generic tflearn function:
print("Dense1 layer biases:")
with model.session.as_default():
print(tflearn.variables.get_value(dense1_vars[1]))

###### It is also possible to retrieve a layer’s weights through its attributes W and b.

In [ ]:
# Get variable's value, using model `get_weights` 
method:
print("Dense2 layer weights:")
print(model.get_weights(dense2.W))

In [ ]:
# Or using generic tflearn function:
print("Dense2 layer biases:")
with model.session.as_default():
print(tflearn.variables.get_value(dense2.b))